In [1]:
# Step 1 – Install Flask and ngrok for Colab
!pip install flask flask-ngrok


In [2]:
import sqlite3

# Connect to SQLite database (or create if it doesn’t exist)
conn = sqlite3.connect('url_shortener.db', check_same_thread=False)
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS urls (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    short_code TEXT UNIQUE,
    original_url TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    access_count INTEGER DEFAULT 0
)
''')
conn.commit()

print("✅ Database initialized successfully!")


✅ Database initialized successfully!


In [3]:
import string, random

def generate_short_code(length=6):
    chars = string.ascii_letters + string.digits
    return ''.join(random.choices(chars, k=length))

print("Example code →", generate_short_code())


Example code → CeULdx


In [4]:
from flask import Flask, request, redirect, render_template_string
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)

HTML_PAGE = '''
<!DOCTYPE html>
<html>
<head>
    <title>Kaif’s URL Shortener</title>
    <style>
        body {font-family:Arial;background:#f4f4f4;margin:50px;}
        .box {background:#fff;padding:30px;border-radius:12px;width:500px;margin:auto;
              box-shadow:0 0 10px #ccc;}
        input[type=text]{width:100%;padding:10px;margin:10px 0;border-radius:8px;
              border:1px solid #ccc;}
        button {background:#007bff;color:white;border:none;padding:10px 20px;
              border-radius:8px;cursor:pointer;}
    </style>
</head>
<body>
<div class="box">
    <h2>🔗 URL Shortener</h2>
    <form method="POST">
        <input type="text" name="long_url" placeholder="Enter a long URL" required>
        <button type="submit">Shorten</button>
    </form>
    {% if short_url %}
        <p>✅ Short URL: <a href="{{ short_url }}" target="_blank">{{ short_url }}</a></p>
    {% endif %}
</div>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    short_url = None
    if request.method == 'POST':
        long_url = request.form['long_url']
        cursor.execute("SELECT short_code FROM urls WHERE original_url=?", (long_url,))
        row = cursor.fetchone()
        if row:
            short_code = row[0]
        else:
            short_code = generate_short_code()
            cursor.execute("INSERT INTO urls (short_code, original_url) VALUES (?, ?)",
                           (short_code, long_url))
            conn.commit()
        short_url = request.host_url + short_code
    return render_template_string(HTML_PAGE, short_url=short_url)

@app.route('/<short_code>')
def redirect_to_url(short_code):
    cursor.execute("SELECT original_url, access_count FROM urls WHERE short_code=?",
                   (short_code,))
    row = cursor.fetchone()
    if row:
        cursor.execute("UPDATE urls SET access_count=? WHERE short_code=?",
                       (row[1] + 1, short_code))
        conn.commit()
        return redirect(row[0])
    return "❌ Invalid URL", 404

print("🚀 Ready to run!")


🚀 Ready to run!


In [6]:
# Start the Flask server (via ngrok for Colab)
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            

In [7]:
from google.colab import files
files.download('url_shortener.db')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>